<a href="https://colab.research.google.com/github/FerAleUtnSantaFe/EjerciciosMonitores/blob/master/tutorial_llms_groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade groq

In [ ]:
import os
from groq import Groq
from google.colab import userdata

# Configurar la API Key de Groq (reemplazar con tu clave)
GROQ_API_KEY = userdata.get('GROQ_API_KEY')  # <-- Reemplazá con tu clave real
client = Groq(api_key=GROQ_API_KEY)

modelo = "llama-3.3-70b-versatile"


## 1. Ejemplo básico de un prompt

In [ ]:
prompt = "¿Cuál es la capital de Argentina?"

respuesta = client.chat.completions.create(
    model=modelo,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(respuesta.choices[0].message.content)


## 2. Práctica: Límite de tokens

In [ ]:
prompt = "Contame una historia muy larga sobre un dragón y un robot."

for max_tokens in [50, 100, 300]:
    print(f"\n--- Máximo de tokens: {max_tokens} ---")
    respuesta = client.chat.completions.create(
        model=modelo,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens
    )
    print(respuesta.choices[0].message.content)


## 3. Práctica: Uso de temperature

In [ ]:
prompt = "Completá la frase: En una noche oscura y silenciosa, el gato se subió al techo y..."

temperaturas = [0.0, 0.3, 0.7, 1.0, 2.0]

for temp in temperaturas:
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=temp,
        max_tokens=60
    )

    print(f"\n--- Temperature {temp} ---\n{response.choices[0].message.content}")

## 4. Práctica: Uso de top-k y top-p

In [ ]:
prompt = "Inventá un chiste corto sobre programadores."

respuesta = client.chat.completions.create(
    model=modelo,
    messages=[{"role": "user", "content": prompt}],
    top_p=0.9
)
print(respuesta.choices[0].message.content)


## 6. Técnicas de prompting

In [ ]:
# System prompt
response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": "Sos un profesor universitario que responde de forma clara, concisa y con ejemplos."
        },
        {
            "role": "user",
            "content": "Explicá qué es una red neuronal."
        }
    ]
)

print(response.choices[0].message.content)


In [ ]:
# Role prompting
response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "user",
            "content": "Actuá como un historiador experto. ¿Cómo afectó la Revolución Francesa al resto del mundo?"
        }
    ]
)

print(response.choices[0].message.content)


In [ ]:
# Contextual prompting
contexto = "La empresa X fabrica bicicletas eléctricas con diseño ecológico."
pregunta = "¿Qué produce la empresa X?"
print(client.chat.completions.create(model=modelo, messages=[{"role": "user", "content": contexto + "\n" + pregunta}]).choices[0].message.content)



In [ ]:
# Zero-shot
print(client.chat.completions.create(model=modelo, messages=[{"role": "user", "content": "Clasificá el siguiente texto como positivo o negativo: 'Me encantó el producto.'"}]).choices[0].message.content)



In [ ]:
# One-shot
response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": "Sos un asistente que transforma frases informales en versiones formales, manteniendo el mismo significado."
        },
        {
            "role": "user",
            "content": """Ejemplo:
Frase informal: Che, ¿me podés dar una mano con esto?
Frase formal: ¿Serías tan amable de ayudarme con esto?

Nueva frase:
Frase informal: Pintó irnos temprano del laburo.
Frase formal:"""
        }
    ],
    temperature=0.3,
)

print(response.choices[0].message.content)

In [ ]:
# Few-shot
response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": "Sos un modelo que clasifica reseñas de películas como POSITIVA, NEUTRAL o NEGATIVA. Respondé solo con la categoría correspondiente."
        },
        {
            "role": "user",
            "content": """Clasificá las siguientes reseñas:
Reseña: "Una obra maestra visual y emocional."
Sentimiento: POSITIVA

Reseña: "La película fue... meh. Ni buena ni mala, simplemente aburrida."
Sentimiento: NEUTRAL

Reseña: "No la recomiendo. Mal actuada y con un guion pobre."
Sentimiento: NEGATIVA

Reseña: "Una experiencia cinematográfica intensa que te deja pensando."
Sentimiento:"""
        }
    ],
    temperature=0.2,
)

print(response.choices[0].message.content)

In [ ]:
# System, Role and Contextual prompting
response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": "Sos un profesor de programación que responde con ejemplos simples y claros."
        },
        {
            "role": "user",
            "content": """
Actuá como si estuvieras dando una clase de introducción a estudiantes que no saben programar.

Contexto:
Queremos explicar qué es una variable en Python.

Explicá el concepto con un ejemplo.
"""
        }
    ]
)

print(response.choices[0].message.content)

In [ ]:
# Step-back prompting
# Prompt SIN step-back
# sin_stepback = client.chat.completions.create(
#     model="llama3-8b-8192",
#     messages=[
#         {
#             "role": "user",
#             "content": "Dame una idea de negocio innovadora para resolver problemas de transporte urbano."
#         }
#     ]
# )

# print("🟡 Resultado SIN Step-back:\n")
# print(sin_stepback.choices[0].message.content)

# Prompt CON step-back prompting
con_stepback = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "user",
            "content": """
Antes de darme una idea de negocio, pensá:
1. ¿Cuáles son los principales problemas actuales del transporte urbano?
2. ¿Qué necesidades no están bien cubiertas?
3. ¿Qué soluciones tecnológicas podrían aplicarse?
Ahora sí, proponé una idea de negocio innovadora y explicá por qué puede funcionar.
"""
        }
    ]
)

print("\n🟢 Resultado CON Step-back:\n")
print(con_stepback.choices[0].message.content)


In [ ]:
# Chain of Thought (CoT)
print(client.chat.completions.create(model=modelo, messages=[{"role": "user", "content": "Si Juan tiene 3 manzanas y compra 2 más, ¿cuántas tiene ahora? Explicá paso a paso."}]).choices[0].message.content)



In [ ]:
# Self-consistency

# Prompt con Chain of Thought
prompt = """Resolvé este problema paso a paso:
Si un tren sale de Córdoba a las 8:00 am y viaja a 80 km/h, ¿a qué hora llega a Rosario, que está a 320 km de distancia?"""

respuestas = []

# Generamos 5 respuestas diferentes
for i in range(5):
    res = client.chat.completions.create(
        model="llama3-8b-8192",
        temperature=0.7,
        messages=[{"role": "user", "content": prompt}]
    )
    respuestas.append(res.choices[0].message.content)

# Mostrar todas las respuestas
for i, r in enumerate(respuestas, start=1):
    print(f"🔁 Razonamiento {i}:\n{r}\n{'-'*60}")

## 7. Generar código en JavaScript

In [ ]:
prompt = "Escribí una función en JavaScript que calcule el factorial de un número."

respuesta = client.chat.completions.create(
    model=modelo,
    messages=[{"role": "user", "content": prompt}]
)
print(respuesta.choices[0].message.content)


## 8. Explicar código en Python

In [ ]:
codigo = '''def contar_palabras(texto):
    palabras = texto.split()
    return len(palabras)'''

prompt = f"Explicá qué hace este código en Python:\n\n{codigo}"

respuesta = client.chat.completions.create(
    model=modelo,
    messages=[{"role": "user", "content": prompt}]
)
print(respuesta.choices[0].message.content)


## 9. Traducir de JavaScript a Python

In [ ]:
js_code = "function sumar(a, b) { return a + b; }"
prompt = f"Traducí este código JavaScript a Python:\n\n{js_code}"

respuesta = client.chat.completions.create(
    model=modelo,
    messages=[{"role": "user", "content": prompt}]
)
print(respuesta.choices[0].message.content)


## 10. Devolver JSON con schema

In [ ]:
schema = {
  "type": "object",
  "properties": {
    "nombre": {"type": "string"},
    "precio": {"type": "number"},
    "stock": {"type": "integer"},
    "categoria": {"type": "string"}
  },
  "required": ["nombre", "precio", "stock", "categoria"]
}

prompt = f"Generá un producto de ejemplo que cumpla con este schema JSON:\n\n{schema}"

respuesta = client.chat.completions.create(
    model=modelo,
    messages=[{"role": "user", "content": prompt}]
)
print(respuesta.choices[0].message.content)
